# Biolog acs-2 screen

In [9]:
%matplotlib inline
import os
import sys
import tifffile as tiff
import pandas as pd
import matplotlib
# For visualising
#matplotlib.use('MacOSX') 
# For saving
#matplotlib.use('Agg') 
matplotlib.use('MacOSX') #TkAgg
import matplotlib.pyplot as plt

import numpy as np
import itertools as IT
import time
import glob
import skimage
from skimage import io as skio
from skimage import color

import cPickle
import gzip

import datetime
from multiprocessing import Pool, Manager
from multiprocessing.sharedctypes import Value, Array
from ctypes import Structure, c_double, c_int

import re

sys.path.append("./src")
from utilities import *

#plt.interactive(True)
matplotlib.rcParams.update({'font.size': 12})
np.set_printoptions(precision=3)

In [10]:
os.chdir("/Users/Povilas/Dropbox/Projects/Metformin_Celegans_Biolog")
sourceloc = "/Users/Povilas/Dropbox/Projects/Metformin_Celegans_Biolog"

# os.chdir("/home/pnorv/Dropbox/Projects/Metformin_TF_acs-2/")
# sourceloc = "home/pnorv/Dropbox/Projects/Metformin_TF_acs-2/"

odir="."

In [4]:
filedata=[]
header=['Replicate_folder','Folder','File']
filedata.append(header)

allfiles=glob.glob('./Rep*/*/*.tif')

for fl in allfiles:
    rep,plate,fln=fl.split('/')[1:]
    filedata.append([rep,plate,fln])
    #print rep, fld, fln
    
writecsv(filedata,'Allfiles.csv')

## Collect images

In [8]:
#filelist=pd.read_csv('./Allfiles_annotated.csv')
oimages="Images_HV.pkl"
#iimagesgz="Images.pkl.gz"
#oimagesgz="Images.pkl.gz"

In [ ]:
settings={'colordepth':'uint8','savetype':'HV'}
args=[(fl,settings,) for fl in allfiles]

results=multiprocessor(4,collector_M,args)

fls,imagesarray=zip(*results)

imagesindex={fl:fli for fli,fl in enumerate(fls) }

print("Saving images: {}".format(time.ctime()))
print "Stacking..."
images=np.stack(imagesarray, axis=0)
print "Done!"

f = open(oimages, "w")
#f = gzip.open(oimagesgz, "w")
print "Writing..."
cPickle.dump((imagesindex,images), f)
f.close()
os.system("gzip {}".format(oimages))
print("Saving images complete!")

## Save a random subset of all images for testing

In [82]:
collectsubset("Images_test",[images],imagesindex,size=50)

100 100
Stacking...
Writing...
Zipping...
Saving images complete!


## Load images

In [11]:
#imagesgz="Images_HV.pkl.gz"
imagesgz="Images_test.pkl.gz"

print time.ctime()
print "Loading images... {}".format(time.ctime())
idata = gzip.open(imagesgz,'rb')
imagesindex, images = cPickle.load(idata)
idata.close()
print "Loading finished! {}".format(time.ctime())
print time.ctime()

Sun Mar  4 15:18:19 2018
Loading images... Sun Mar  4 15:18:19 2018
Loading finished! Sun Mar  4 15:18:27 2018
Sun Mar  4 15:18:27 2018


In [12]:
images.shape

(100, 1040, 1392, 2)

## Segmentation testing

In [13]:
olabels="Segmentation_test.pkl"
otbl="All_results_test.csv"

filesel=imagesindex.keys()[:50]
ofig="Test.pdf"

In [14]:
settings={'fd':10}

args=[(fln,images[imagesindex[fln]],settings) for fln in filesel]

results=multiprocessor(4,labeller5_M,args)

fls, filteredarray, labelsarray, resultsarray = zip(*results)

#Collect and save results
labelsindex={fl:fli for fli,fl in enumerate(fls) }


results_data=[]
res_header=['File','Worm','W_N','W_Sum','W_Mean','B_N','B_Sum','B_Mean']

for fl, respack in zip(fls, resultsarray):
    for row in respack:
        if len(row)>0:
            results_data.append([fl]+row)    

# Save results
results=pd.DataFrame(results_data,columns=res_header)
results.to_csv(otbl)

print("Saving labels: {}".format(time.ctime()))
print "Stacking..."

labels=np.stack(labelsarray, axis=0)
filtered=np.stack(filteredarray, axis=0)
print "Stacking complete!"

print "Writing..."
#f = gzip.open(olabelsgz, "w")
f = open(olabels, "w")
#cPickle.dump((labelsindex,labels), f)
cPickle.dump((labelsindex,filtered,labels), f)
f.close()
print "Zipping..."
os.system("gzip -f {}".format(olabels))
print("Saving labels complete! {}".format(time.ctime()))

/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


IndexError: too many indices for array

In [ ]:
# rndsel=np.random.randint(len(allfiles)-1,size=5)
#filesel=[ allfiles[i] for i in rndsel ]

ncols=2
row=0


fig, axes = plt.subplots(nrows=len(filesel), ncols=2, figsize=(6*ncols,5*len(filesel)), dpi=300)
#fig.suptitle('{}-{}'.format(replicate, plate), fontsize=40)
#plt.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=2, wspace=0.5, hspace=0.1)


for flid, fl in enumerate(filesel):
    print(fl)
    
    rep,fld,fln=fl.split('/')[1:]
    flname,flex=fln.split('.')
    

    #GetRGB data from a collapsed image
    image_raw=images[imagesindex[fl]]
    
    if np.ndim(image_raw)==2:
        #image=color.gray2rgb(image_raw)
        v=image_raw
    else:
        #image=image_raw
        v = color.rgb2hsv(image_raw)[:,:,2]
        
    image_jet=jetimage(v,typeadjust=False)
    labeled_worms=labels[labelsindex[fl]]

    plotcontours(image_jet,axes[row,0],ylabel=flname,title='Normalised intensity',plotcontour=False,plotlabels=False,white=False)
    plotcontours(image,axes[row,1],labeled_worms,title='Segmented',plotlabels=True,white=True)
    row+=1

#fig.tight_layout()
fig.savefig(ofig, bbox_inches='tight')
plt.close(fig)


print time.ctime()



In [15]:
alldeletions = readdel('/Users/Povilas/Dropbox/Projects/2015-Metformin/Biolog_Met_NGM/Celegans/Deletions.csv')
thresholds_man = readman('/Users/Povilas/Dropbox/Projects/2015-Metformin/Biolog_Met_NGM/Celegans/Thresholds_all_3.csv')

otbl = 'Summary_{}_{}.csv'.format('rep56', label)
olabels = 'Segmentation_default.pkl'

/Users/Povilas/Dropbox/Projects/2015-Metformin/Biolog_Met_NGM/Celegans/Deletions.csv
Necessary headers found!
/Users/Povilas/Dropbox/Projects/2015-Metformin/Biolog_Met_NGM/Celegans/Thresholds_all_3.csv
Necessary headers found!


In [16]:
#settings={'fd':10}
args=[]
for fln in imagesindex.keys()[:10]:
    replicate,plate,fl=fln.split('/')[1:]
    flname,flex=fl.split('.')
    
    indx=numerize(re.findall(r'\d+', flname)[-1])
    
    fpat, fname, ftype = filename(fln)
    
    hv=images[imagesindex[fln]]
    
    print replicate, plate, fln, indx
    
    if indx in alldeletions[replicate][plate].keys():
        delworms = alldeletions[replicate][plate][indx]['Worms']  
    else:
        delworms = []
        
    
    if indx in thresholds_man[replicate[3]][plate].keys():
        mthr=thresholds_man[replicate[3]][plate][indx]
    else:
        mthr=''

    args.append((fln,replicate,hv,delworms,mthr,settings))

Rep1 PM4A ./Rep1/PM4A/PM4A_091.tif 91
Rep3 PM2A ./Rep3/PM2A/PM2A_087.tif 87
Rep6 PM2A ./Rep6/PM2A/PM2A_084.tif 84
Rep4 PM3B ./Rep4/PM3B/PM3B_069.tif 69
Rep2 PM2A ./Rep2/PM2A/PM2A_096.tif 96
Rep4 PM1 ./Rep4/PM1/PM1_030.tif 30
Rep2 PM4A ./Rep2/PM4A/PM4A_017.tif 17
Rep3 PM2A ./Rep3/PM2A/PM2A_078.tif 78
Rep5 NGM_Control ./Rep5/NGM_Control/NGM_NoMetf_004.tif 4
Rep3 PM1 ./Rep3/PM1/PM1_055.tif 55


In [ ]:
hv=images[imagesindex[fln]]

fln,imagef,labeled_worms,results_data=labeller4B_Mt((fln,replicate, hv, [], mthr, settings, ''))

FWHM: 0.0173082428497, mu: 0.331277829392, SD: 0.00735013398847


In [21]:
def labeller4B_Mt((fln,replicate, hv, delworms, mthr, settings, q),cthres=0.02,lstep=0.001, dvd=3, dhd=3 , cod=3, sd=10, r14_mu=0.995318, r14_sd=0.427193, r14_c=0.020434, r56_mu=0.995318, r56_sd=0.427193, r56_c=0.03, blobsize=1000, maxsize=1000000):

    for key,val in settings.items():
        exec(key + '=val')
        
    levels = ["{0:.3f}".format(lvl) for lvl in np.arange(0, 1 + lstep, lstep)]
    
    hvdel=wormdel(hv, delworms)
    hvclean = skimage.img_as_float(hvdel)
    
    h=hvclean[:,:,0]
    v=hvclean[:,:,1]
    
    v1D = np.around(h.ravel(), 3)
    ftable = np.array(freqtable(v1D))
    ftable = ftable[np.argsort(ftable[:, 0])]
    X, Y, mu, sd = fitgauss(ftable[:, 0], ftable[:, 1])
    # What hue threshold o use


    if replicate in ['Rep1','Rep2','Rep3','Rep4']:
        hthr=mu * r14_mu + sd * r14_sd + r14_c
        #hthr=mu * 0.995318 + sd * 0.427193 + 0.020434
    else:
        hthr=mu * r56_mu + sd * r56_sd + r56_c
        #hthr=mu * 0.995318 + sd * 0.427193 + 0.03


    #Use manual value or estimate
    hthres = mthr if mthr!='' and mthr!=0 else hthr


    # Filtering
    dh = skimage.img_as_float(rank.median(h, disk(dhd)))# / 255.0
    dv = skimage.img_as_float(rank.median(v, disk(dvd)))# / 255.0
    # np.max(dh)
    # hmax=np.percentile(h,95)

    hf = dh.copy()
    hf[hf < hthres] = 0
    hf = hf / hthres
    
    # plt.imshow(hf)

    comb_r = hf * dv
    comb = opening(comb_r, selem=disk(3))

    markers = np.zeros_like(comb)
    # Mark background
    markers[comb == 0] = 1
    markers[comb > cthres] = 2

    labeled_worms=waterseg(markers,sd,blobsize)

    #Data collection step
    #In Velocity background is summed in only one channel!
    results_data=getdistributions(v,labeled_worms,levels)
    
    #q.put(fln)
    
    return [fln,imagef,labeled_worms,results_data]




In [ ]:
results=multiprocessor(4,labeller4B_Mt,args)

fls, filteredarray, labelsarray, resultsarray = zip(*results)

#Collect and save results
labelsindex={fl:fli for fli,fl in enumerate(fls) }

results_data=[]
lstep=0.001
levels = ["{0:.3f}".format(lvl) for lvl in np.arange(0, 1 + lstep, lstep)]
res_header=['File','Worm']+levels

for fl, respack in zip(fls, resultsarray):
    for row in respack:
        if len(row)>0:
            results_data.append([fl]+row)
            

In [53]:
# Save results
results=pd.DataFrame(results_data,columns=res_header)
results.to_csv(otbl)

print("Saving labels: {}".format(time.ctime()))
print "Stacking..."

labels=np.stack(labelsarray, axis=0)
filtered=np.stack(filteredarray, axis=0)
print "Stacking complete!"

print "Writing..."
#f = gzip.open(olabelsgz, "w")
f = open(olabels, "w")
#cPickle.dump((labelsindex,labels), f)
cPickle.dump((labelsindex,filtered,labels), f)
f.close()
print "Zipping..."
os.system("gzip -f {}".format(olabels))
print("Saving labels complete! {}".format(time.ctime()))

AttributeError: 'module' object has no attribute 'as_type_float'

In [ ]:




header = ['Replicate', 'Plate', 'Well', 'File', 'Worm'] + levels
data.insert(0, header)
writecsv(data, ofname, '\t')

#Save thresholds
header = ['Replicate', 'Plate', 'Well', 'File', 'Worm'] + ['Mu','SD','Manual_t','Estimated_t']
thresholds.insert(0, header)
writecsv(thresholds, otname, '\t')